In [ ]:
import asyncio
import re
import os
import pandas as pd
from openai import AsyncOpenAI
from dotenv import load_dotenv
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, stop_after_delay,
)

# Load environment variables
load_dotenv()

class ZeroShot:
    def __init__(self, api_key=None, base_url=None, model="gpt-4o-mini", system_prompt=None, temperature=0.2):
        # Use environment variables if no parameters are provided
        self.api_key = api_key or os.getenv("API_KEY")
        self.base_url = base_url or os.getenv("BASE_URL")
        self.model = model
        self.system_prompt = system_prompt or "Based on the user's query, generate Python code using `matplotlib.pyplot` and 'seaborn' to create the requested plot. If data is needed, use data from the data path provided. Ensure the code is outputted within the format ```python...```."
        self.temperature = temperature
        # Initialize OpenAI client
        self.client = AsyncOpenAI(
            api_key=self.api_key,
            base_url=self.base_url
        )

    def describe_data(self, data_path):

        return str(data_path)
    

    @retry(wait=wait_random_exponential(min=0.02, max=1), stop=(stop_after_delay(3) | stop_after_attempt(30)))
    async def call_openai_api(self, system_prompt, user_content):
        try:
            response = await self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_content}
                ],
                temperature=self.temperature
            )
            
            # Extracting response content and token usage
            message_content = response.choices[0].message.content
            prompt_tokens = response.usage.prompt_tokens
            completion_tokens = response.usage.completion_tokens
            total_tokens = response.usage.total_tokens
            
            return {
                "response": message_content,
                "prompt_tokens": prompt_tokens,
                "completion_tokens": completion_tokens,
                "total_tokens": total_tokens
            }
            
        except Exception as e:
            print(f"API call failed with error: {e}. Retrying...")
            raise e
            
    async def call_openai_api(self, user_query, data_description):
        # Retry indefinitely until successful
        while True:
            try:
                # Concatenate the data description and user query
                # full_query = f"Data Description: {data_description}\nUser Query: {user_query}"
                full_query = f"Data Path: {data_description}\nUser Query: {user_query}"
                
                # print(full_query)
                
                # Call OpenAI API to get the response
                response = await self.client.chat.completions.create(
                    model=self.model,
                    messages=[
                        {"role": "system", "content": self.system_prompt},
                        {"role": "user", "content": full_query}
                    ],
                    temperature=self.temperature
                )
                prompt_tokens = response.usage.prompt_tokens
                completion_tokens = response.usage.completion_tokens
                total_tokens = response.usage.total_tokens
            

                # Extract the code content from the response
                response_text = response.choices[0].message.content
                match = re.search(r'```python\n(.*?)```', response_text, flags=re.DOTALL)
                print({
                    "prompt_tokens": prompt_tokens,
                    "completion_tokens": completion_tokens,
                    "total_tokens": total_tokens
                })
                
                if match:
                    return match.group(1).strip()
                else:
                    return None
            except Exception as e:
                print(f"API call failed with error: {e}. Retrying...")

    async def get_code_content(self, user_query, data_path_list, img_file_path=None):
        # print(f'사용하는 모델 명 : {self.model}')        
        data_description = []
        # data_description = ''
        if data_path_list:
            for data_path in data_path_list:
                root_path = os.path.abspath(os.path.dirname(os.curdir))
                data_path = os.path.normpath(os.path.join(root_path, data_path))
            data_description.append(data_path)
        else:
            data_description =  None
        # Call API and return the generated code
        code_content = await self.call_openai_api(user_query, data_description)
        return code_content

In [18]:
dataset_path = r"..\..\dataset\matplotbench_data.csv"

dataset_df = pd.read_csv(dataset_path)

In [19]:
sample = [10, 20, 30, 40, 50]
query_list = dataset_df.loc[sample,'simple_instruction']

In [22]:
zs = ZeroShot()
# user_query = "Scatter plot을 그리고 싶어. X축과 Y축 데이터를 받아서 시각화해줘."
for user_query in query_list:
    code_content = await zs.get_code_content(user_query, None)
# print("🔹 생성된 코드:\n", code_content)

사용하는 모델 명 : gpt-4o-mini
{'prompt_tokens': 430, 'completion_tokens': 486, 'total_tokens': 916}
사용하는 모델 명 : gpt-4o-mini
{'prompt_tokens': 206, 'completion_tokens': 298, 'total_tokens': 504}
사용하는 모델 명 : gpt-4o-mini
{'prompt_tokens': 240, 'completion_tokens': 493, 'total_tokens': 733}
사용하는 모델 명 : gpt-4o-mini
{'prompt_tokens': 279, 'completion_tokens': 440, 'total_tokens': 719}
사용하는 모델 명 : gpt-4o-mini
{'prompt_tokens': 319, 'completion_tokens': 475, 'total_tokens': 794}


Prompt Tokens: 294.8

Completion Tokens: 438.4

Total Tokens: 733.2